In [11]:
import torch

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available! Using GPU.")
else:
    print("CUDA is not available. Using CPU.")


CUDA is available! Using GPU.


In [4]:
from ucimlrepo import fetch_ucirepo 

# fetch dataset 
estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition = fetch_ucirepo(id=544) 
  
# data (as pandas dataframes) 
X = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.features 
y = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.targets.copy() 
  
# metadata 
print(estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.metadata) 
  
# variable information 
print(estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.variables) 


{'uci_id': 544, 'name': 'Estimation of Obesity Levels Based On Eating Habits and Physical Condition ', 'repository_url': 'https://archive.ics.uci.edu/dataset/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition', 'data_url': 'https://archive.ics.uci.edu/static/public/544/data.csv', 'abstract': 'This dataset include data for the estimation of obesity levels in individuals from the countries of Mexico, Peru and Colombia, based on their eating habits and physical condition. ', 'area': 'Health and Medicine', 'tasks': ['Classification', 'Regression', 'Clustering'], 'characteristics': ['Multivariate'], 'num_instances': 2111, 'num_features': 16, 'feature_types': ['Integer'], 'demographics': ['Gender', 'Age'], 'target_col': ['NObeyesdad'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2019, 'last_updated': 'Tue Sep 10 2024', 'dataset_doi': '10.24432/C5H31Z', 'creators': [], 'intro_paper': {'ID': 358, 'type': 

In [5]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
import torch.nn as nn

from pyro.infer import MCMC, NUTS
from pyro.infer import Predictive

c:\Users\peter\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
categorical_features = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC','CALC','MTRANS']
# One-hot encode categorical features, dropping the first category
X_encoded = pd.get_dummies(X, columns=categorical_features, drop_first=True)
X_encoded = X_encoded.astype({col: int for col in X_encoded.select_dtypes('bool').columns})

# Convert X_encoded to a PyTorch tensor
X_tensor = torch.tensor(X_encoded.values, dtype=torch.float32)


In [7]:
y['NObeyesdad'] = y['NObeyesdad'].astype('category').cat.codes

# Convert to tensor
y_tensor = torch.tensor(y['NObeyesdad'].to_numpy(), dtype=torch.long)

In [8]:
x_tr, x_te, y_tr, y_te = train_test_split(X_tensor, y_tensor, test_size=0.3, random_state=42)

In [59]:
class BNN(PyroModule):
    def __init__(self, in_dim=1, out_dim=1, hid_dim=10, n_hid_layers=2, prior_scale=5., device = 'cpu'):
        super().__init__()
        self.in_dim = in_dim
        self.device = device
        self.activation = nn.ReLU()  # could also be ReLU or LeakyReLU
        assert in_dim > 0 and out_dim > 0 and hid_dim > 0 and n_hid_layers > 0  # make sure the dimensions are valid

        # Define the layer sizes and the PyroModule layer list
        self.layer_sizes = [in_dim] + n_hid_layers * [hid_dim] + [out_dim]
        layer_list = [PyroModule[nn.Linear](self.layer_sizes[idx - 1], self.layer_sizes[idx]) for idx in
                      range(1, len(self.layer_sizes))]
        self.layers = PyroModule[torch.nn.ModuleList](layer_list)#.to(self.device)

        for layer_idx, layer in enumerate(self.layers):
            layer.weight = PyroSample(dist.Normal(0., torch.tensor(prior_scale * np.sqrt(2 / self.layer_sizes[layer_idx]), dtype=torch.float, device = device)).expand(
                [self.layer_sizes[layer_idx + 1], self.layer_sizes[layer_idx]]).to_event(2))
            layer.bias = PyroSample(dist.Normal(0., torch.tensor(prior_scale, dtype=torch.float, device=device)).expand([self.layer_sizes[layer_idx + 1]]).to_event(1))

        # for layer in self.layers:
        #     layer.weight = layer.weight.to(self.device)  # Move weight tensor
        #     layer.bias = layer.bias.to(self.device)  # Move bias tenso
        
    def forward(self, x, y=None):
        x = x.to(device)
        x = x.reshape(-1, self.in_dim)

        x = self.activation(self.layers[0](x))  # input --> hidden
        for layer in self.layers[1:-1]:
            x = self.activation(layer(x))  # hidden --> hidden
        x = self.layers[-1](x).squeeze()  # hidden --> output
        x = torch.softmax(x, dim=1) # softmax activation
        y = y.to(device)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Categorical(x), obs=y)
        return x

In [62]:
# Define Hamiltonian Monte Carlo (HMC) kernel
# NUTS = "No-U-Turn Sampler" (https://arxiv.org/abs/1111.4246), gives HMC an adaptive step size
#in_dim=1, out_dim=1, hid_dim=10, n_hid_layers=5, prior_scale=5.
x_tr, y_tr = x_tr.to(device), y_tr.to(device)
x_te, y_te = x_te.to(device), y_te.to(device)
model = BNN(in_dim=x_tr.shape[1], out_dim=7, hid_dim=10, n_hid_layers=2, prior_scale=5., device = device)#.to(device)

In [63]:
# define model and data

# define MCMC sampler
nuts_kernel = NUTS(model, jit_compile=True)
mcmc = MCMC(nuts_kernel, num_samples=250, warmup_steps=500)
mcmc.run(x_tr, y_tr)

Warmup:   0%|          | 0/750 [00:00, ?it/s]c:\Users\peter\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyro\poutine\subsample_messenger.py:70: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor(0.0, device=self.device)
Sample: 100%|██████████| 750/750 [1:58:57,  9.52s/it, step size=2.93e-04, acc. prob=0.961]


In [64]:
post_samples = mcmc.get_samples()

keys = list(post_samples.keys())
print(keys)

post_samples[keys[1]].shape

['layers.0.bias', 'layers.0.weight', 'layers.1.bias', 'layers.1.weight', 'layers.2.bias', 'layers.2.weight']


torch.Size([250, 10, 23])

In [67]:
import pickle

# Save dictionary to a pickle file
with open('post_samples.pkl', 'wb') as f:
    pickle.dump(post_samples, f)

# Load dictionary from the pickle file
with open('post_samples.pkl', 'rb') as f:
    loaded_samples = pickle.load(f)

In [68]:
loaded_samples

{'layers.0.bias': tensor([[-0.1837, -1.1126, -0.5612,  ..., -0.2912,  1.3319,  5.4971],
         [-0.0365, -1.0210, -0.5311,  ..., -0.3331,  1.3475,  5.5114],
         [-0.0238, -1.0140, -0.5473,  ..., -0.3932,  1.3438,  5.4972],
         ...,
         [ 0.5947, -1.2018, -0.6721,  ..., -0.9122,  0.9699,  4.6841],
         [ 0.4834, -1.1944, -0.6413,  ..., -1.0111,  0.9822,  4.6518],
         [ 0.5320, -1.1777, -0.6394,  ..., -0.9437,  0.9775,  4.6194]],
        device='cuda:0'),
 'layers.0.weight': tensor([[[-1.6514e+00, -1.1698e+00,  5.2780e-01,  ...,  2.2376e-01,
           -3.0341e+00,  2.2060e+00],
          [ 1.8739e+00,  1.4889e-02, -2.1047e+00,  ...,  1.2718e+00,
            7.8635e-01, -9.6376e-01],
          [ 6.5406e-01,  1.1728e+00,  2.9917e+00,  ..., -3.7824e-01,
            2.0104e+00,  3.1188e+00],
          ...,
          [ 2.8865e+00,  5.0704e-01, -1.8589e+00,  ..., -1.2812e+00,
           -2.3845e-02,  1.4113e+00],
          [-2.2488e+00, -5.7536e-01, -8.5153e-01,  ...

In [17]:
predictive = Predictive(model=model, posterior_samples=mcmc.get_samples())
preds = predictive(x_te)['obs']

mse = nn.MSELoss()
mse(preds, y_te)

c:\Users\peter\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([634])) that is different to the input size (torch.Size([50, 634])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(7.1215)

In [24]:
#preds is 50x634
preds

tensor([[4, 0, 3,  ..., 2, 5, 2],
        [0, 5, 5,  ..., 5, 1, 4],
        [6, 3, 6,  ..., 5, 3, 6],
        ...,
        [4, 1, 0,  ..., 5, 0, 3],
        [2, 3, 1,  ..., 6, 0, 2],
        [2, 1, 3,  ..., 2, 4, 2]])

In [28]:
relative_frequencies = torch.zeros(7, 634)
for j in range(preds.shape[1]):
    # Count the occurrences of each label in column j
    label_counts = torch.bincount(preds[:, j], minlength=7)
    # Calculate the relative frequency by dividing by the total number of predictions (50)
    relative_frequencies[:, j] = label_counts.float() / preds.shape[0]

relative_frequencies

tensor([[0.0800, 0.0400, 0.1800,  ..., 0.1400, 0.1800, 0.0200],
        [0.1000, 0.2600, 0.2000,  ..., 0.1600, 0.0600, 0.3000],
        [0.1600, 0.0400, 0.1200,  ..., 0.2000, 0.1600, 0.1200],
        ...,
        [0.2000, 0.2200, 0.1600,  ..., 0.1400, 0.3000, 0.2800],
        [0.1000, 0.0600, 0.0600,  ..., 0.1200, 0.1200, 0.0200],
        [0.2600, 0.0000, 0.1000,  ..., 0.1400, 0.0600, 0.0200]])